In [13]:
from pandas import read_csv

df = read_csv('../csv/data.csv', index_col='id')
df[
    (52 < df.latitude_dd) \
        & (df.latitude_dd < 54) \
            & (79 < df.longitude_dd) \
                & (df.longitude_dd < 81)]['population'].sum()

170034

In [11]:
125+103+701+347

1276